## Requirements

In [95]:
# Keras
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense
# Util
import numpy as np
import matplotlib.pyplot as plt
# Excell
import openpyxl as xl

## Processing data

### Input leyer size

In [96]:
# loading an existing workbook
wb = xl.load_workbook('Geology Input for Petar Updated April 8th.xlsx')
# Select the worksheet
items_sheet = wb['Sheet1']
# make an values only iterator over the items from row 5 to 15 and col 2 to 12.
# turn it into a tuple. Take only the first element, the second one is empty
terain_properties = list(items_sheet.iter_cols(min_row=1, max_col=2, min_col=2, values_only=True))[0]
terain_properties = list(map(lambda x: x.strip(), terain_properties))

num_of_properties = len(terain_properties)

### Getting training and testing data

In [97]:
data_sheet = wb['Sheet2']

river_channel_cols = tuple(data_sheet.iter_cols(min_row=4, max_row=9, min_col=3, max_col=8, values_only=True))
river_channel_cols = [tuple(xi for xi in x if xi is not None) for x in river_channel_cols]

flood_plane_cols = tuple(data_sheet.iter_cols(min_row=11, max_row=14, min_col=3, max_col=8, values_only=True))
flood_plane_cols = [tuple(xi for xi in x if xi is not None) for x in flood_plane_cols]

river_delta_cols = tuple(data_sheet.iter_cols(min_row=16, max_row=22, min_col=3, max_col=8, values_only=True))
river_delta_cols = [tuple(xi for xi in x if xi is not None) for x in river_delta_cols]

alluvial_fan_cols = tuple(data_sheet.iter_cols(min_row=24, max_row=26, min_col=3, max_col=8, values_only=True))
alluvial_fan_cols = [tuple(xi for xi in x if xi is not None) for x in alluvial_fan_cols]

desert_dune_cols = tuple(data_sheet.iter_cols(min_row=28, max_row=30, min_col=3, max_col=8, values_only=True))
desert_dune_cols = [tuple(xi for xi in x if xi is not None) for x in desert_dune_cols]

In [98]:
def prepareStrings(items):
    acc = []
    for item_list in items:
        if len(acc) == 0:
            acc = list(map(lambda x: [x.strip()] ,item_list))
            continue
        
        current_acc = []
        for item in item_list:
            for line in acc:
                current_acc.append(line + [item.strip()])
        acc = current_acc
    return np.asarray(acc)

river = prepareStrings(river_channel_cols)
flood_plane = prepareStrings(flood_plane_cols)
river_delta = prepareStrings(river_delta_cols)
alluvial_fan = prepareStrings(alluvial_fan_cols)
desert_dune = prepareStrings(desert_dune_cols)

print(river.shape)
print(flood_plane.shape)
print(river_delta.shape)
print(alluvial_fan.shape)
print(desert_dune.shape)


(432, 6)
(32, 6)
(756, 6)
(24, 6)
(12, 6)


In [99]:
train_strings = np.concatenate((river, flood_plane, river_delta, alluvial_fan, desert_dune))
X_train = np.zeros((1256, num_of_properties),dtype='float32')
for index, st in enumerate(train_strings):
    for prop in st:
        X_train[index][terain_properties.index(prop)] = 1.0

y_river = np.ones(river.shape[0])*0
y_flood_plane = np.ones(flood_plane.shape[0])*1
y_river_delta = np.ones(river_delta.shape[0])*2
y_alluvial_fan = np.ones(alluvial_fan.shape[0])*3
y_desert_dune = np.ones(desert_dune.shape[0])*4

y_train = np.concatenate((y_river, y_flood_plane, y_river_delta, y_alluvial_fan, y_desert_dune))

y_train = to_categorical(y_train, num_classes=5)

## Model

In [100]:
model = Sequential()

model.add(Dense(250, activation="relu", input_shape=(num_of_properties,)))
# model.add(Dense(512,activation="relu"))
model.add(Dense(5, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

history = model.fit(X_train, y_train, epochs=20, validation_data=(X_train, y_train))

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 250)               18500     
                                                                 
 dense_12 (Dense)            (None, 5)                 1255      
                                                                 
Total params: 19,755
Trainable params: 19,755
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/20
40/40 [==============================] - 1s 11ms/step - loss: 0.7903 - accuracy: 0.8607 - val_loss: 0.3074 - val_accuracy: 0.9459
Epoch 2/20
40/40 [==============================] - 0s 8ms/step - loss: 0.2002 - accuracy: 0.9459 - val_loss: 0.1233 - val_accuracy: 0.9522
Epoch 3/20
40/40 [==============================] - 0s 9ms/step - loss: 0.0906 - accuracy: 0.9682 - val_loss: 0.0594 - val_accuracy: 0.9936
Epoch 4/20
40/40 [==

In [101]:
model.save('./models/newest.model')

INFO:tensorflow:Assets written to: ./models/newest.model\assets
